In [30]:
from pathlib import Path
from typing import Dict, List, Union, Iterable, Tuple  
import json
import pandas as pd
def load_parquet_dict(data_folder: str) -> dict[str, pd.DataFrame]:
    p = Path(data_folder)
    with open(p / "_dtype_book.json", "r", encoding="utf-8") as f:
        book = json.load(f)

    out = {}
    for fp in sorted(p.glob("*.parquet")):
        if fp.name.startswith("_"):
            continue
        name = fp.stem
        df = pd.read_parquet(fp)
        spec = book.get(name)
        if spec:
            df = df.astype(spec)
        else:
            raise Exception(f"{name} in dtype_book.json is {spec}")
        out[name.lower()] = df
    return out

dfs = load_parquet_dict("data/cleaned_result")
dfs

{'assignments':            W6KEY  REVISION         TIMECREATED        TIMEMODIFIED       TASK  \
 0        8340677         4 2022-01-18 11:46:14 2022-01-18 14:50:40        NaN   
 1        8340691         6 2022-01-18 14:49:07 2022-01-19 15:01:06        NaN   
 2        8343748         4 2022-01-25 08:45:51 2022-02-01 13:49:52        NaN   
 3        8344077         4 2022-01-25 08:45:49 2022-02-01 12:21:58        NaN   
 4        8344080         5 2022-01-25 08:45:54 2022-02-01 10:00:12        NaN   
 ...          ...       ...                 ...                 ...        ...   
 5743416  3028307         7 2020-06-23 14:50:25 2020-07-23 11:23:01  3797218.0   
 5743417  3034692         6 2021-06-28 16:03:05 2021-08-18 15:55:18  4178269.0   
 5743418  3052118         8 2020-06-26 08:01:46 2020-08-06 11:38:05  3805222.0   
 5743419  3060903        11 2017-11-08 13:13:27 2018-01-12 13:07:33  2758601.0   
 5743420  3064265         5 2021-07-02 16:06:21 2021-07-13 13:01:48  4184898.0   
 

In [ ]:
# if you would like to import the original data, use the following code
# from merge_table import join_csv_by_keywords
# original_dfs, filemap = join_csv_by_keywords(
#     folder="data/click/",
#     given_list={"ASSIGNMENTS": ["ASSIGNMENTS"],
#                 "DEPARTMENT":["DEPARTMENT"],
#                 "DISTRICTs":["DISTRICTS"],
#                 "ENGINEERS":["ENGINEERS"],
#                 "EQUIPMENT":["EQUIPMENT"],
#                 "TASK_STATUSES":["TASK_STATUSES"],
#                 "TASK_TYPES":["TASK_TYPES"],
#                 "TASKS":["TASKS"]
#                 },
                
#     return_filemap=True
# )

In [31]:
pd.set_option('display.max_columns', None)

### - How long does each type of work typically take?

Here’s a recap of what we discussed in our previous meeting.
1. Work Analysis
Duration by Work Type
- How long does each type of work typically take?
Potential System Gaming
- Are there instances where work is completed unusually fast?
- Could crews be performing the work and logging it later, affecting reported duration?
Impact on Actual Duration
- How does this behavior influence the accuracy of recorded durations?
Duration Expectations
- What should be the standard or expected duration for each type of work?
Distribution and Outliers
- What percentage of work falls within the expected duration?
- What do the outliers suggest—are they anomalies, errors, or signs of inefficiency?

Planned vs. Actual Duration
- How closely does actual duration align with what was planned?
2. Scheduling Effectiveness
Crew Scheduling
- Are we assigning work to crews in a balanced and efficient way?
Workload Appropriateness
- Are crews receiving the right amount of work—too much, too little, or just right?
Schedule Feasibility
- Is the schedule realistic and achievable given current resources and constraints?
3. Operational Efficiency
Planned vs. Completed Work
- Does the work completed match what was planned for the day?
Productivity Assessment
- How productive are crews based on completed work versus expectations?
Improvement Opportunities
- Where are the gaps or inefficiencies?
- What changes could improve performance or accuracy?


#### W6TASKS – Work Order/Task data 
#### W6ASSIGNMENTS – Defines what ‘Task’ was scheduled, when and to who.
#### W6DEPARTMENT – Part of hierarchy, We are concerned with ‘SDGE-GAS’
#### W6REGIONS – Part of hierarchy, We are concerned with ‘%-GAS’
#### W6DEPARTMENT – Part of hierarchy, this defines the districts.
#### W6ENGINEERS – Define Crew Shells and People. We normally schedule work to crew shells which are manned by people.
#### W6EQUIPMENT – Defines equipment that is a part of the inspection order, probably not useful for our efforts. 
#### W6TASK_STATUSES – Status codes for orders.
#### W6TASK_TYPES – Work types, defines what the order is concerned with. We are focusing on those where the name contains ‘%SDG’
